# Transfer Learning

## Load Libraries and Function Declarations



In [1]:
import importlib
import sys
import os
import numpy as np
import time
from pathlib import Path
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader

from torch.utils import data
from torchvision import transforms

sys.path.append(os.path.join(os.getcwd(), ".."))

from distiller import apputils
import ai8x

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

kws20 = importlib.import_module("datasets.kws20-horsecough")

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support, roc_auc_score, roc_curve


import librosa
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# FIX SEED FOR REPRODUCIBILITY
seed = 69
torch.manual_seed(seed)

def rescale(audio, min_val=-1,max_val=1):
    sig = audio
    mean = np.average(sig)

    sig = sig-mean # REMOVE DC COMPONENT

    sig_max = np.max(sig)
    sig_min = np.min(sig)

    if sig_max >= np.abs(sig_min):
        sig_scaled = sig/sig_max
    else:
        sig_scaled = sig/np.abs(sig_min)

    return sig_scaled

def rescale2(audio, min_val=-1,max_val=1):
    scaler = MinMaxScaler(feature_range=(min_val,max_val))
    audio = audio.reshape(-1,1)
    scaler.fit(audio)
    scaled = np.array(scaler.transform(audio))
    
    return scaled[:,0]


def plot_confusion(y_true, y_pred, classes):
    cf_matrix = confusion_matrix(y_true = y_true, y_pred = y_pred, labels =list(range(len(classes))))
    print(cf_matrix)

def plot_roc_curve(true_y, y_prob):
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.plot([0,1], [0,1], 'r--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

def custom_dataloader(data, train_idx, val_idx, test_idx, batch_size = 2048):
    data_file = data
    x = np.asarray(data_file[0][test_idx])
    y = np.squeeze(np.asarray(data_file[1][test_idx]))
    test_set = TensorDataset(torch.Tensor(x),torch.Tensor(y).to(dtype =torch.int64))
    test_loader = DataLoader(test_set,batch_size=batch_size, num_workers=4, pin_memory=True)

    x = np.asarray(data_file[0][val_idx])
    y = np.squeeze(np.asarray(data_file[1][val_idx]))
    val_set = TensorDataset(torch.Tensor(x),torch.Tensor(y).to(dtype =torch.int64))
    val_loader = DataLoader(val_set,batch_size=batch_size, num_workers=4, pin_memory=True)

    x = np.asarray(data_file[0][train_idx])
    y = np.squeeze(np.asarray(data_file[1][train_idx]))
    train_set = TensorDataset(torch.Tensor(x),torch.Tensor(y).to(dtype =torch.int64))
    train_loader = DataLoader(train_set,batch_size=batch_size, num_workers=4, pin_memory=True)

    return train_loader,val_loader,test_loader

def freeze_layer(layer):
    for p in layer.parameters():
        p.requires_grad = False

def count_params(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    print(params)
    return params

    

## Prepare Checkpoints Folder

In [2]:
# MODEL NAME AND CHECKPOINT FOLDER
name = 'equine'
classes = ['a_combined', 'a_cough']

indexer = 0
while True:
    model_name = name + '_' + str(indexer)
    checkpoint_dir = './checkpoints/'+model_name+'/'
    indexer += 1

    if os.path.exists(checkpoint_dir) is False:
        break

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

print('Model Name: ', model_name)
print('Classes: ', classes)
print('Checkpoint Dir: ', checkpoint_dir)

# CALCULATE WEIGHTS
raw_data_path = Path("../data/KWS_EQUINE/raw/")
class_file_count = {}
class_dirs = [d for d in raw_data_path.iterdir() if d.is_dir() and (d.stem != "__combinedkws") and (d.stem != "__combined_others") and (d.stem != "__human_cough")and (d.stem != "__human_cough_v2")]

for d in class_dirs:
    print('Path: ',d)
    class_file_count[d] = len(list(d.iterdir()))

min_file_count = float(min(class_file_count.values()))


class_weights = []
for d in class_dirs:
    class_file_count[d] = min_file_count / class_file_count[d]
    print(f"{d.stem}: {round(class_file_count[d], 7)}")
    class_weights.append(round(class_file_count[d], 7))
print('Weights: ',class_weights)

Model Name:  equine_27
Classes:  ['a_combined', 'a_cough']
Checkpoint Dir:  ./checkpoints/equine_27/
Path:  ..\data\KWS_EQUINE\raw\a_combined
Path:  ..\data\KWS_EQUINE\raw\a_cough
a_combined: 0.8236659
a_cough: 1.0
Weights:  [0.8236659, 1.0]


## Generate processed dataset

In [3]:
# NOTE: Smart compressed file creation
# Change class dicts of main Dataloader class
train_batch_size = 1024
train_loader, val_loader, test_loader, _ = apputils.get_data_loaders(
    kws20.KWS_HORSE_TF_get_datasets, ("../data", False), train_batch_size, 4, validation_split=0.2)

print(f"Dataset sizes:\n\ttraining={len(train_loader.sampler)}\n\tvalidation={len(val_loader.sampler)}\n\ttest={len(test_loader.sampler)}")


No key `noise_var` in input augmentation dictionary!  Using defaults: [Min: 0., Max: 1.]
No key `shift` in input augmentation dictionary! Using defaults: [Min:-0.1, Max: 0.1]
No key `strech` in input augmentation dictionary! Using defaults: [Min: 0.8, Max: 1.3]

Processing train...
Class a_combined not found in data

Processing test...
Class a_combined not found in data
Dataset sizes:
	training=20311
	validation=5078
	test=2907


## Horse Cough Dataset

In [4]:
# HORSE COUGH DATA PATH
inf_paths = ["../data/KWS_EQUINE/inference/horse_cough/","C:/Users/J_C/Desktop/DATASETS_N/horse_cough_stable/"]

files = []
for path_root in inf_paths:
            path_file = os.listdir(path_root)
            for p in path_file:
                files.append(path_root+p)

inf_count = len(files)
y_true_inf = np.ones(inf_count)

print('\n Horse Cough File Count: ',inf_count)


 Horse Cough File Count:  137


## Load Reference Model and Train Parameters

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    cpu = False
else:
     device = torch.device('cpu')
     cpu = True
     
ai8x.set_device(device=85, simulate=False, round_avg=False)

# mod = importlib.import_module("models.ai85net-kws20-v3")
# model = mod.AI85KWS20Netv3(num_classes=21, num_channels=128, dimensions=(128, 1), bias=False)

mod = importlib.import_module("models.ai85net-equine")
model = mod.AI85EQUINE()


# WEIGHTS OF REFERENCE MODEL
# model, compression_scheduler, optimizer, start_epoch = apputils.load_checkpoint(
#             model, "../logs/kws20_original/qat_best.pth.tar")

model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=1e-05)
ms_lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15000], gamma=0.2)


criterion = torch.nn.CrossEntropyLoss(weight=torch.Tensor(class_weights))
criterion.to(device)

qat_policy = {
    'start_epoch': 20,
    'weight_bits': 8
    }


with open(checkpoint_dir+'/model_params.txt','w+') as f:
    f.write('Running on device: {}'.format(torch.cuda.get_device_name()))
    f.write('\nNumber of Model Params: {}'.format(count_params(model)))
    f.write('\nOptimizer: {}'.format(optimizer))
    f.write('\nLoss Function: {}'.format(criterion))
    f.write('\nMLR Scheduler: {}'.format(ms_lr_scheduler))
    f.write('\nQAT: {}'.format(qat_policy))
    f.write('\nMODEL: {}'.format(model))


print('Running on device: {}'.format(torch.cuda.get_device_name()))
print(f'Number of Model Params: {count_params(model)}')
print('Optimizer: \n',optimizer)
print('Loss Function: \n',criterion)
print('\nMLR Scheduler: {}'.format(ms_lr_scheduler))
print('QAT: \n',qat_policy)
print(model)


Configuring device: MAX78000, simulate=False.
1016576
Running on device: NVIDIA GeForce RTX 2070 SUPER
1016576
Number of Model Params: 1016576
Optimizer: 
 Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.0003
    lr: 0.0003
    maximize: False
    weight_decay: 0.0001
)
Loss Function: 
 CrossEntropyLoss()

MLR Scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x0000025BE14AF610>
QAT: 
 {'start_epoch': 20, 'weight_bits': 8}
AI85EQUINE(
  (drop): Dropout(p=0.2, inplace=False)
  (voice_conv1): FusedConv1dReLU(
    (activate): ReLU(inplace=True)
    (op): Conv1d(128, 100, kernel_size=(1,), stride=(1,), bias=False)
    (calc_out_shift): OutputShiftSqueeze()
    (calc_weight_scale): One()
    (scale): Scaler()
    (calc_out_scale): OutputScale()
    (quantize_weight): Empty()
    (quantize_bias): Empty()
    (clamp_weight): Empty()
    (clamp_bias): Empty()
    (quantize): Empty()
    (clamp

## Train the Model

In [6]:
# best_acc = 0
# best_qat_acc = 0
# for epoch in range(0, num_epochs):
#     if epoch > 0 and epoch == qat_policy['start_epoch']:
#         print('QAT is starting!')
#         # Fuse the BN parameters into conv layers before Quantization Aware Training (QAT)
#         ai8x.fuse_bn_layers(model)

#         # Switch model from unquantized to quantized for QAT
#         ai8x.initiate_qat(model, qat_policy)

#         # Model is re-transferred to GPU in case parameters were added
#         model.to(device)
#     running_loss = []
#     train_start = time.time()
#     model.train()
#     for idx, (inputs, target) in enumerate(train_loader):
#         inputs = inputs.to(device)
#         target = target.to(device)
#         optimizer.zero_grad()
        
#         model_out = model(inputs)
        
#         loss = criterion(model_out, target)
#         loss.backward()
#         optimizer.step()
        
#         running_loss.append(loss.cpu().detach().numpy())

#     mean_loss = np.mean(running_loss)
#     train_end = time.time()
#     print("Epoch: {}/{}\t LR: {}\t Train Loss: {:.4f}\t Dur: {:.2f} sec.".format(
#         epoch+1, num_epochs, ms_lr_scheduler.get_lr(), mean_loss, (train_end-train_start)))
    
#     model.eval()
#     acc = 0.
#     acc_weight = 0
#     y_true = []
#     y_pred = []
#     with torch.no_grad():
#         for inputs, target in test_loader:
#             inputs = inputs.to(device)
#             target = target.to(device)
#             model_out = model(inputs)
#             target_out = torch.argmax(model_out, dim=1)
            
#             y_pred.extend(target_out.cpu().numpy())
#             y_true.extend(target.cpu().numpy())
            
#             tp = torch.sum(target_out == target)
#             acc_batch = (tp / target_out.numel()).detach().item()
#             acc += target_out.shape[0] * acc_batch
#             acc_weight += target_out.shape[0]
            
#         total_acc = 100 * (acc / acc_weight)
#         if epoch == qat_policy['start_epoch']: best_acc = 0
#         if total_acc > best_acc:
#             best_acc = total_acc
#             checkpoint_extras = {'current_top1': best_acc,
#                                  'best_top1': best_acc,
#                                  'best_epoch': epoch}
#             model_name = 'ai85net_kws_equine'
#             model_prefix = f'{model_name}' if epoch < qat_policy['start_epoch'] else (f'qat_{model_name}')
#             apputils.save_checkpoint(epoch, model_name, model, optimizer=optimizer,
#                                      scheduler=None, extras=checkpoint_extras,
#                                      is_best=True, name=model_prefix,
#                                      dir='.')
#             print(f'Best model saved with accuracy: {best_acc:.2f}%')
            
#         print('\t\t Test Acc: {:.2f}'.format(total_acc))
#         print("\t\tConfusion:")
#         plot_confusion(y_true, y_pred, classes)
#     ms_lr_scheduler.step()

In [7]:
num_epochs = 1000
best_acc = 0
best_qat_acc = 0
best_loss = 0
best_epoch = 0

train_acc = []
val_acc = []
test_acc = []
inf_acc = []

train_loss = []
val_loss = []
test_loss = []

running_class_acc = []

continue_train = True
epoch = 0

while(continue_train and epoch < num_epochs):
    average_acc = 0
    train_start = time.time()
    if epoch > 0 and epoch == qat_policy['start_epoch']:
        print('QAT is starting!')
        # Fuse the BN parameters into conv layers before Quantization Aware Training (QAT)
        ai8x.fuse_bn_layers(model)
        # Switch model from unquantized to quantized for QAT
        ai8x.initiate_qat(model, qat_policy)
        # Model is re-transferred to GPU in case parameters were added
        model.to(device)
        best_acc = 0

    ############ TRAIN SECTION ############
    model.train()
    acc_b = []   
    y_pred_train = []
    y_true_train = []
    running_loss = []
    for idx, (inputs, target) in enumerate(train_loader):
        # temp_var = inputs.cpu().numpy()
        # print(np.min(temp_var),np.max(temp_var))
        # input()
        inputs = inputs.to(device)
        target = target.type(torch.int64)
        target = target.to(device)
        optimizer.zero_grad()
        
        model_out = model(inputs)
        target_out = torch.argmax(model_out, dim=1)
        
        y_pred_train.extend(target_out.cpu().numpy())
        y_true_train.extend(target.cpu().numpy())
        
        tp = torch.sum(target_out == target)
        acc_b.extend([(tp / target_out.numel()).detach().item()])

        loss = criterion(model_out, target)     
        loss.backward()
        optimizer.step()
        
        running_loss.append(loss.cpu().detach().numpy())
    
    total_acc = np.mean(acc_b)*100
    mean_loss = np.mean(running_loss)

    # TRAIN ACCURACY / TRAIN LOSS
    train_acc.append(total_acc)
    train_loss.append(mean_loss)
    average_acc += total_acc*0.5


    model.eval()
    with torch.no_grad():
        ############ VALIDATION SECTION ############
        acc_b = []
        y_pred_val = []
        y_true_val = []
        running_v_loss = []
        for inputs, target in val_loader:
            inputs = inputs.to(device)
            target = target.to(device)
            model_out = model(inputs)
            target_out = torch.argmax(model_out, dim=1)
            
            y_pred_val.extend(target_out.cpu().numpy())
            y_true_val.extend(target.cpu().numpy())
            
            tp = torch.sum(target_out == target)
            acc_b.extend([(tp / target_out.numel()).detach().item()])

            loss = criterion(model_out, target)
            running_v_loss.append(loss.cpu().detach().numpy())

        total_acc = np.mean(acc_b)*100
        mean_loss = np.mean(running_v_loss)
        
        # VALIDATION ACCURACY / VALIDATION LOSS
        val_acc.append(total_acc)
        val_loss.append(mean_loss)
        average_acc += total_acc*0.5

        ############ TEST SECTION ############
        acc_b = []
        y_pred_test = []
        y_true_test = []
        running_t_loss = []
        time_start = time.time()
    
        for inputs, target in test_loader:
            inputs = inputs.to(device)
            target = target.to(device)
            model_out = model(inputs)
            target_out = torch.argmax(model_out, dim=1)

            y_true_test.extend(target.cpu().numpy())
            y_pred_test.extend(target_out.cpu().numpy())

            tp = torch.sum(target_out == target)
            acc_b.extend([(tp / target_out.numel()).detach().item()])

            loss = criterion(model_out, target)
            running_t_loss.append(loss.cpu().detach().numpy())

        total_acc = np.mean(acc_b)*100
        mean_loss = np.mean(running_t_loss)
        
        # VALIDATION ACCURACY / VALIDATION LOSS
        test_acc.append(total_acc)
        test_loss.append(mean_loss)

        ############ HORSE COUGH SECTION ############
        y_pred_inf = []
        acc_b = 0
        for counter,f in enumerate(files):
            try:
                # CONVERT EACH AUDIO FILE TO A 128X128 ARRAY TO TENSOR
                data_sq = np.zeros(128)
                data, sr = librosa.load(f,sr = 16000)
                data = rescale2(data,min_val=-1,max_val=1)
                data = librosa.util.fix_length(data,size=int(128*128))
                for index in range(0,len(data),128):
                    data_row = data[index:index+128]
                    data_sq = np.vstack((data_sq,data_row))
                data_sq = data_sq[1:129]
                data_sq = data_sq.transpose()
                data_sq = np.expand_dims(data_sq, axis=0)
                inputs = torch.from_numpy(data_sq.astype(np.float32))  
                inputs.to(torch.uint8)

                ############ INFERENCE SECTION ############
                inputs = inputs.to(device)
                model_out = model(inputs)
                target_out = torch.argmax(model_out, dim=1)
                class_output = target_out.detach().item()
                if class_output == 1: acc_b += 1/inf_count*100
            
            except Exception as e:
                print(e)
                
        inf_acc.append(acc_b)

    ############ CLASS ACCURACY ############
    class_acc = np.zeros(len(classes))
    for class_num,class_type in enumerate(classes):
        class_count_train = y_true_train.count(class_num)
        class_count_val = y_true_val.count(class_num)

        for t_idx, targ_val in enumerate(y_true_train):
            if targ_val == y_pred_train[t_idx] and targ_val == class_num:
                class_acc[class_num] += 1/class_count_train*10

        for t_idx, targ_val in enumerate(y_true_val):
            if targ_val == y_pred_val[t_idx] and targ_val == class_num:
                class_acc[class_num] += 1/class_count_val*90
        
    running_class_acc.append(class_acc)
    
    train_end = time.time()
    print('---------------------------------------------')
    print("\n\n Epoch: {}/{} \tLR: {} \tDur: {:.2f} sec".format(epoch+1, num_epochs, ms_lr_scheduler.get_lr() , (train_end-train_start)))

    ############ CONFUSION MATRIX ############   
    print("\n TRAIN - Confusion Matrix: ")
    plot_confusion(y_true_train, y_pred_train, classes)
    print("\n VAL - Confusion Matrix: ")
    plot_confusion(y_true_val, y_pred_val, classes)
    print("\n TEST - Confusion Matrix: ")
    plot_confusion(y_true_test, y_pred_test, classes)
    
    ############ ACC and LOSS ############  
    print('\nTrain Acc : ', train_acc[-1])
    print('Train Loss : ', train_loss[-1])
    print('Val Acc : ', val_acc[-1])
    print('Val Loss : ', val_loss[-1])
    print('Test Acc : ', test_acc[-1])
    print('Test Loss : ', test_loss[-1])
    print('Inference Acc : ', inf_acc[-1])

    ############ PLOTS ############
    if (epoch%5 == 0 or epoch==num_epochs-1) and epoch > 0:
        best_epoch = checkpoint_extras['best_epoch']
        
        plt.figure(figsize=(20,10),dpi=300)
        plt.title(model_name)
        plt.subplot(1,2,1)
        plt.plot(np.asarray(running_class_acc)[:,0],color='orange')
        plt.plot(np.asarray(running_class_acc)[:,1],color='yellow')
        plt.plot(test_acc,color = 'black')
        plt.plot(val_acc, color ='green')
        plt.plot(train_acc, color = 'red')
        plt.plot(inf_acc, color ='blue')
        plt.stem(best_epoch,train_acc[best_epoch])
        plt.legend([classes[0],classes[1],'Test','Validation','Train','Horse Cough','Checkpoint'])
        # plt.legend(['Test','Validation','Train','Horse Cough','Checkpoint'])

        plt.title('Accuracy: {:.2f}'.format(train_acc[best_epoch]))
        plt.xlabel('Epochs')
        plt.ylabel('Value')

        plt.subplot(1,2,2)
        plt.plot(test_loss, color = 'black')
        plt.plot(val_loss,color='green')
        plt.plot(train_loss,color='red')
        plt.stem(best_epoch,train_loss[best_epoch])
        plt.legend(['Test','Validation','Train','Checkpoint'])
        plt.title('Loss: {:.2f}'.format(train_loss[best_epoch]))
        plt.ylim(0,1)
        plt.xlabel('Epochs')
        plt.ylabel('Value')

        plt.savefig(checkpoint_dir+model_name+'.png')
        plt.clf()
        plt.cla()
        plt.close()
    
    ############ SAVE CHECKPOINT ############   
    if average_acc > best_acc:
        best_acc = average_acc
        checkpoint_extras = {'best_ave_acc': best_acc,
                                'best_epoch': epoch}
        
        model_prefix = f'{model_name}' if epoch < qat_policy['start_epoch'] else (f'qat_{model_name}')
        apputils.save_checkpoint(epoch, model_name, model, optimizer=optimizer,
                                    scheduler=None, extras=checkpoint_extras,
                                    is_best=True, name=model_prefix,
                                    dir=checkpoint_dir)

        # PLOT CONFUSION MATRIX AND STAT MEASURES ON TRAIN
        conf_mat_train = confusion_matrix(y_true_train, y_pred_train)
        cm_display_train = ConfusionMatrixDisplay(confusion_matrix = conf_mat_train, display_labels = classes)
        p_train,r_train,f1_train,_= precision_recall_fscore_support(y_true_train, y_pred_train, average=None)
        cm_display_train.plot(cmap= 'Blues',colorbar=False, values_format = 'd')
        plt.title('Preicison: ({:.2f} {:.2f})   Recall: ({:.2f} {:.2f})   F1-Score: ({:.2f} {:.2f})'.format(p_train[0],p_train[1],r_train[0],r_train[1],f1_train[0],f1_train[1]))
        plt.savefig(checkpoint_dir+model_name+'_cm_TRAIN.png')
        plt.clf()
        plt.cla()
        plt.close()

        # PLOT CONFUSION MATRIX AND STAT MEASURES ON VALIDATION
        conf_mat_val = confusion_matrix(y_true_val, y_pred_val)
        cm_display_val = ConfusionMatrixDisplay(confusion_matrix = conf_mat_val, display_labels = classes)
        p_val,r_val,f1_val,_= precision_recall_fscore_support(y_true_val, y_pred_val, average=None)
        cm_display_val.plot(cmap= 'Blues',colorbar=False, values_format = 'd')
        plt.title('Preicison: ({:.2f} {:.2f})   Recall: ({:.2f} {:.2f})   F1-Score: ({:.2f} {:.2f})'.format(p_val[0],p_val[1],r_val[0],r_val[1],f1_val[0],f1_val[1]))
        plt.savefig(checkpoint_dir+model_name+'_cm_VAL.png')
        plt.clf()
        plt.cla()
        plt.close()

        # PLOT CONFUSION MATRIX AND STAT MEASURES ON TEST
        conf_mat_test = confusion_matrix(y_true_test, y_pred_test)
        cm_display_test = ConfusionMatrixDisplay(confusion_matrix = conf_mat_test, display_labels = classes)
        p_test,r_test,f1_test,_= precision_recall_fscore_support(y_true_test, y_pred_test, average=None)
        cm_display_test.plot(cmap= 'Blues',colorbar=False, values_format = 'd')
        plt.title('Preicison: ({:.2f} {:.2f})   Recall: ({:.2f} {:.2f})   F1-Score: ({:.2f} {:.2f})'.format(p_test[0],p_test[1],r_test[0],r_test[1],f1_test[0],f1_test[1]))
        plt.savefig(checkpoint_dir+model_name+'_cm_TEST.png')
        plt.clf()
        plt.cla()
        plt.close()


        print(f' --------------------------------------------------------->   Model Checkpoints Saved with Mean Accuracy : {best_acc:.2f}%')
    
        ############ STOP TRAINING ############ 
        if epoch > num_epochs*0.75 and best_acc > 95:
            print('--------------------------------------------------------->   Ending Training, Best Checkpoint Found')
            continue_train = False
            break
    
    ms_lr_scheduler.step()
    epoch += 1

---------------------------------------------


 Epoch: 1/1000 	LR: [0.0003] 	Dur: 12.05 sec

 TRAIN - Confusion Matrix: 
[[10898   132]
 [ 9006   275]]

 VAL - Confusion Matrix: 
[[2797    0]
 [2281    0]]

 TEST - Confusion Matrix: 
[[1689    0]
 [1218    0]]

Train Acc :  54.99962866306305
Train Loss :  0.6897743
Val Acc :  55.0808310508728
Val Loss :  0.6749587
Test Acc :  58.09432864189148
Test Loss :  0.6674533
Inference Acc :  0
 --------------------------------------------------------->   Model Checkpoints Saved with Mean Accuracy : 55.04%
---------------------------------------------


 Epoch: 2/1000 	LR: [0.0003] 	Dur: 10.40 sec

 TRAIN - Confusion Matrix: 
[[9404 1626]
 [5954 3327]]

 VAL - Confusion Matrix: 
[[2119  678]
 [ 947 1334]]

 TEST - Confusion Matrix: 
[[1233  456]
 [ 494  724]]

Train Acc :  62.705495059490204
Train Loss :  0.66159666
Val Acc :  67.97519445419312
Val Loss :  0.63338184
Test Acc :  67.2434707482656
Test Loss :  0.6269303
Inference Acc :  54.744525

## INFERENCE ON HORSE COUGH

In [8]:
# print('Processing Inference on Custom Dataset \n')

# mod = importlib.import_module("models.ai85net-equine")
# model = mod.AI85EQUINE()
# try:
#     model, compression_scheduler, optimizer, start_epoch = apputils.load_checkpoint(model, checkpoint_dir+'qat_'+model_name+'_best.pth.tar')
# except:
#     model, compression_scheduler, optimizer, start_epoch = apputils.load_checkpoint(model, checkpoint_dir+model_name+'_best.pth.tar')
    
# # model, compression_scheduler, optimizer, start_epoch = apputils.load_checkpoint(model, './checkpoints/equine_12/qat_equine_12_best.pth.tar')
# model = model.to(device)



# class_paths = {'horse_cough_internet': "../data/KWS_EQUINE/inference/horse_cough/",
#                'horse_cough_stable': "C:/Users/J_C/Desktop/DATASETS_N/horse_cough_stable/"}

# sample_count = False
# y_true_inf = []
# y_pred_inf = []



# classes = list(class_paths.keys())
# time_start = time.time()
# with torch.no_grad():
#     for class_ix,inf_path in enumerate(list(class_paths.values())):
#         files = os.listdir(inf_path)
#         file_count = len(files)
#         inferences = []
        
#         random.shuffle(files)
        
#         if sample_count < file_count and type(sample_count) == int : files = files[0:sample_count]

#         for counter,f in enumerate(files):
#             try:
#                 # CONVERT EACH AUDIO FILE TO A 128X128 ARRAY
#                 data_sq = np.zeros(128)
#                 data, sr = librosa.load(inf_path+f,sr = 16000)
#                 data = rescale2(data,min_val=-1,max_val=1) 
#                 data = librosa.util.fix_length(data,size=int(128*128))
#                 for index in range(0,len(data),128):
#                     data_row = data[index:index+128]
#                     data_sq = np.vstack((data_sq,data_row))
#                 data_sq = data_sq[1:129]
#                 data_sq = data_sq.transpose()
                
#                 # CONVERT ARRAY TO TENSOR
#                 data_sq = np.expand_dims(data_sq, axis=0)
#                 inputs = torch.from_numpy(data_sq.astype(np.float32))  
#                 inputs.to(torch.uint8)
#                 ############ INFERENCE SECTION ############
#                 inputs = inputs.to(device)
#                 model_out = model(inputs)
#                 target_out = torch.argmax(model_out, dim=1)
#                 class_output = target_out.detach().item()

#                 y_true_inf.append(class_ix)
#                 y_pred_inf.append(class_output)

                
#                 print('Remaining: ',(file_count-counter),'\tTrue:', class_ix,'\tOutput:', class_output)
#             except Exception as e:
#                 print(e)

#      #print('\t\t Test Acc: {:.2f}'.format(total_acc))
   

# time_end = time.time()

# print('Inference Finished in {:2f} seconds'.format(time_end-time_start))

# conf_mat_train = confusion_matrix(y_true_inf, y_pred_inf)
# cm_display_train = ConfusionMatrixDisplay(confusion_matrix = conf_mat_train, display_labels = classes)
# p_train,r_train,f1_train,_= precision_recall_fscore_support(y_true_inf, y_pred_inf, average=None)
# cm_display_train.plot(cmap= 'Blues',colorbar=False, values_format = 'd')


# plt.title('Preicison: ({:.2f} {:.2f})   Recall: ({:.2f} {:.2f})   F1-Score: ({:.2f} {:.2f})'.format(p_train[0],p_train[1],r_train[0],r_train[1],f1_train[0],f1_train[1]))

# fname = model_name+'_'+str(classes[0])+'_'+str(classes[1])+'_cm_inference_horse.png'
# fname = model_name+'_cm_HORSE.png'
# plt.savefig(checkpoint_dir+fname)
# plt.clf()
# plt.cla()
# plt.close()

## INFERENCE ON WHOLE DATASET

In [9]:
# print('Processing Inference on Training Dataset \n')

# # LOAD DATASET FILE
# # data_file = torch.load(processed_dir+model_name+'.pt') # (data, class, type)
# processed_dir = '../data/KWS_EQUINE/processed'
# data_file =torch.load(processed_dir+'/dataset2.pt')

# y_true_inf = data_file[1].cpu().numpy()
# y_pred_inf = []
# file_count = len(y_true_inf)

# time_start = time.time()
# model.eval()

# with torch.no_grad():
#     for counter,val in enumerate((data_file[0])):
#         val =val[None,:]
#         inputs = val.to(torch.float)
#         ############ INFERENCE SECTION ############
#         inputs = inputs.to(device)
#         model_out = model(inputs)
#         target_out = torch.argmax(model_out, dim=1)
        
#         class_output = target_out.detach().item()

#         print('Remaining: ',(file_count-counter),'\tTrue:', y_true_inf[counter],'\tOutput:', class_output)
#         y_pred_inf.append(class_output)

#      #print('\t\t Test Acc: {:.2f}'.format(total_acc))
   

# time_end = time.time()

# print('Inference Finished in {:2f} seconds'.format(time_end-time_start))

# conf_mat_train = confusion_matrix(y_true_inf, y_pred_inf)
# cm_display_train = ConfusionMatrixDisplay(confusion_matrix = conf_mat_train, display_labels = classes)
# p_train,r_train,f1_train,_= precision_recall_fscore_support(y_true_inf, y_pred_inf, average=None)
# cm_display_train.plot(cmap= 'Blues',colorbar=False, values_format = 'd')


# plt.title('Preicison: ({:.2f} {:.2f})   Recall: ({:.2f} {:.2f})   F1-Score: ({:.2f} {:.2f})'.format(p_train[0],p_train[1],r_train[0],r_train[1],f1_train[0],f1_train[1]))

# fname = model_name+'_cm_WHOLE.png'
# plt.savefig(checkpoint_dir+fname)
# plt.clf()
# plt.cla()
# plt.close()